# Simplified Multi-Domain Assessment

Intention of this section is to assess the usefulness of *non prioritised* weighting in multi domain assessment. Effectively applying MTMF with individual / subsets of metrics across domains. 

## 1-Metric Matrix

$$
H = [H_0, H_1, \cdots, H_M]^T = 
\left[ 
\begin{array}{cccc}
1 & 0 & \cdots  & 0 \\
0 & 1 & \cdots  & 0 \\
0 & 0 & \cdots  & 1
\end{array}
\right]$$

$$ \begin{align}
  \Delta T_{ix} &= \frac{\sum_{j\neq x}\left( \overline{T_{i,j}}^{\forall t}\right)}{N-1} \\
  \Delta T_{ix}^- &= \frac{\sum_{j\neq x} \Delta T_{ij}}{N-1}
\end{align}
$$

The Performance ($P$) and False Positive Performance ($P^-$) of the system can then be described as

$$ H . 
\left[
\begin{array}{c}
\Delta T_{ix}^0 \\
\Delta T_{ix}^1 \\
\cdots \\
\Delta T_{ix}^M
\end{array}
\right] = P$$

Results generated using Combined Comparisons Run with `max_emphasis` of 2 and `max_sum` of 1

In [125]:
%matplotlib inline
import os

import pandas as pd
import itertools
from collections import defaultdict
import bounos.Analyses.Weight as Weight


observer = 'Bravo'
target = 'Alfa'
n_nodes = 6
n_metrics = 9

results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Control-2016-02-25-13-38-42"
outlier_path = os.path.join(results_path, "outliers.h5")


## Machine Learning Discussion

From a machine learning perspective, the challenge with MLing this classification / regression is language. 

In the below headed DataFrame, the "Metrics" can be considered ML "Features", with the "Var" being the "Label (or target)". But it's not that simple; as these labels only apply in the case of the relationships between $N_{i, i\neq0}$ and $N_0$ (where $N_0$ is the malicious node).

As such it doesn't make sense to use the raw metric results as ML training data. Rather using the grey-weighted value with singular / basic weights (i.e. no non binary prioritisation)

In [128]:
with pd.get_store(results_path + '.h5') as store:
    tf = store.trust
    tx = store.tx
    acc = store.trust_accessories

In [145]:
#tx[tx.ID.str.contains('_')]
tf.PLR

var                              run  observer  t   target 
CombinedBadMouthingPowerControl  0    Alfa      1   Alfa      NaN
                                                    Bravo       0
                                                    Charlie     0
                                                    Delta       0
                                                    Echo        0
                                                    Foxtrot     0
                                                2   Alfa      NaN
                                                    Bravo       0
                                                    Charlie     0
                                                    Delta       0
                                                    Echo        0
                                                    Foxtrot     0
                                                3   Alfa      NaN
                                                    Bravo       0
                

In [119]:
outlier_weights = Weight.build_outlier_weights(outlier_path, observer=observer, target=target, 
                                        n_metrics=n_metrics, signed=False)

outlier_weights.sort_index()

/CombinedTrust_0_4
/CombinedTrust_1_4
/CombinedTrust_2_4
/CombinedTrust_3_4


var                                                             CombinedBadMouthingPowerControl  \
ADelay ARXP ATXP RXThroughput PLR TXThroughput INDD INHD Speed                                    
0      0    0    0            0   0            0    0    1                             1.536689   
                                                         1                             2.678020   
                                                         1                             5.093316   
                                                         1                             3.228756   
                                                    1    0                             1.214175   
                                                         0                             1.331672   
                                                         0                             5.600897   
                                                         0                             0.873078   
                                               1    0    0                             0.940407   
                                                         0                             3.212840   
                                                         0                             2.556087   
                                                         0                             1.691036   
                                  1            0    0    0                             1.565171   
                                                         0                             0.875971   
                                                         0                             2.040387   
                                                         0                             0.741897   
                 1            0   0            0    0    0                             1.000976   
                                                         0                             1.150225   
                                                         0                             1.377230   
                                                         0                             0.749230   
            1    0            0   0            0    0    0                            14.500000   
                                                         0                            14.000000   
                                                         0                            14.500000   
                                                         0                            14.500000   
       1    0    0            0   0            0    0    0                             2.835919   
                                                         0                             0.325720   
                                                         0                            10.837542   
                                                         0                             0.593956   
1      0    0    0            0   0            0    0    0                             1.531012   
                                                         0                             1.449570   
                                                         0                             0.636476   
                                                         0                             1.639101   

var                                                             CombinedSelfishTargetSelection  \
ADelay ARXP ATXP RXThroughput PLR TXThroughput INDD INHD Speed                                   
0      0    0    0            0   0            0    0    1                            1.196526   
                                                         1                            4.974364   
                                                         1                            5.699470   
                                                         1                            1.363624   
                                             

In [174]:
metric_keys = list(outlier_weights.index.names)
var_keys = list(outlier_weights.keys())
X = pd.DataFrame(outlier_weights.groupby(level=metric_keys).mean().to_records())[metric_keys].astype(int)
Y = pd.DataFrame(outlier_weights.groupby(level=metric_keys).mean().to_records())[var_keys]
outlier_weights.stack(level='var')

ADelay  ARXP  ATXP  RXThroughput  PLR  TXThroughput  INDD  INHD  Speed  var                            
0       0     0     0             0    0             0     0     1      CombinedBadMouthingPowerControl     1.536689
                                                                        CombinedSelfishTargetSelection      1.196526
                                                                        CombinedTrust                       1.709057
                                                                        Shadow                              2.058487
                                                                        SlowCoach                           5.090524
                                                                        CombinedBadMouthingPowerControl     2.678020
                                                                        CombinedSelfishTargetSelection      4.974364
                                                                        Combi

In [92]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=0)

In [ ]:
svm.fit()

In [106]:
len(np.packbits(X.values.astype(int).T))

36

In [107]:
Y.shape

(32, 5)

In [177]:
Y['metric']=map(lambda x: [metric_keys[i] for i,v in enumerate(x) if v], X.values)

In [178]:
Y

,CombinedBadMouthingPowerControl,CombinedSelfishTargetSelection,CombinedTrust,Shadow,SlowCoach,metric
0,3.134195,3.308496,1.669565,3.336074,5.850248,[Speed]
1,2.254955,2.435865,1.195038,1.960323,2.193700,[INHD]
2,2.100092,2.404516,1.409881,1.426872,1.314699,[INDD]
3,1.305856,1.974231,1.111700,1.279051,1.234656,[TXThroughput]
4,1.069415,1.442890,1.025151,1.313753,1.304512,[RXThroughput]
5,14.375000,0.000000,0.000000,0.000000,0.000000,[ATXP]
6,3.648284,2.376358,1.105105,2.826743,3.430264,[ARXP]
7,1.314040,0.918763,1.227195,1.483989,1.480081,[ADelay]
